<a href="https://colab.research.google.com/github/tatooland/keras/blob/master/KERAS_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##define GOOGLE DRIVER downloader

In [0]:
!pip install -U -q PyDrive
import os
from zipfile import ZipFile
from shutil import copy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

def downloadFromGDriver(fileId):
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)
  fileName = fileId + '.zip'
  downloaded = drive.CreateFile({'id': fileId})
  downloaded.GetContentFile(fileName)
  ds = ZipFile(fileName)
  ds.extractall()
  os.remove(fileName)
  print('Extracted zip file ' + fileName)


## define uncompress zip file processing

In [0]:
import os
from zipfile import ZipFile
def uncompressZipFile(fileName):
   ds = ZipFile(fileName)
   ds.extractall()
   os.remove(fileName)
   print("process complete")

##define image iterator

In [0]:
import os
def iteratorImgFolder(folder, callback):
  g = os.walk(folder)
  ary = []
  for path, _, fileList in g:
    for fileName in fileList:
      callback(os.path.join(path, fileName), ary)
  return ary

## define image reader


###download image data from google driver example:

fileId = "1Uy-VRPQlrjfPHkJlZEgxUH-_4vMI1CUn"

downloadFromGDriver(fileId)

uncompressZipFile("seg_pred.zip")

uncompressZipFile("seg_test.zip")

uncompressZipFile("seg_train.zip")

In [0]:
from keras.preprocessing.image import load_img, img_to_array
def readImgAsArray(fileName, ary):
  img = load_img(fileName)
  format_img = img_to_array(img)
  print(format_img.shape)
  ary.extend(format_img)

##define image pre processing

In [0]:
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

def readImgDataAsFlow(train_dir, test_dir, validation_dir, target_size=(150, 150)):
  trainGenerator = ImageDataGenerator(data_format="channels_last")
  testGenerator = ImageDataGenerator(data_format="channels_last")
  validationGenerator = ImageDataGenerator(data_format="channels_last")
  trainFlow = trainGenerator.flow_from_directory(train_dir, class_mode='categorical', target_size=target_size)
  testFlow = testGenerator.flow_from_directory(test_dir, class_mode='categorical', target_size=target_size)
  validationFlow = validationGenerator.flow_from_directory(validation_dir, class_mode='categorical', target_size=target_size)
  return trainFlow, testFlow, validationFlow
  

##define model

###DCGAN

Notes on Implementation
We trained a custom version of the SAGAN model using spectral normalization and self-attention. We used Tensorflow’s tf.keras and Eager execution.

The Generator takes a random vector z and generates 128x128 RGB images. All layers, including dense layers, use spectral normalization. Additionally, the generator uses batch normalization and ReLU activations. Also, it uses self-attention in between middle-to-high feature maps. Like in the original implementation, we placed the attention layer to act on feature maps with dimensions 32x32.

The discriminator also uses spectral normalization (all layers). It takes RGB image samples of size 128x128 and outputs an unscaled probability. It uses leaky ReLUs with an alpha parameter of 0.02. Like the generator, it also has a self-attention layer operating of feature maps of dimensions 32x32.

![](https://sthalles.github.io/assets/advanced_gans/model_architecture.png)


SAGAM model architecture
The goal is to minimize the hinge version of the adversarial loss. To do that, we trained the generator and discriminator in an alternating style using the Adam Optimizer.


![](https://sthalles.github.io/assets/advanced_gans/hinge_gan_loss.png)




[referrence](https://github.com/Goldesel23/DCGAN-for-Bird-Generation/blob/master/traindcgan.py)

###tensorflow code below:
```
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    assert model.output_shape == (None, 7, 7, 256) # Note: None is the batch size

    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    assert model.output_shape == (None, 7, 7, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 14, 14, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 28, 28, 1)

    return model
```

![](https://gluon.mxnet.io/_images/dcgan.png)



```
import time
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np

from keras.models import Sequential
from keras.layers import Conv2D, Conv2DTranspose, Reshape
from keras.layers import Flatten, BatchNormalization, Dense, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


# Here is where we will load the dataset stored in dataset_path. In this script
# we will use the Caltech-UCSD Birds-200-2011 dataset which includes 11788
# images from 200 different birds. We will feed the images without applying
# the provided bounding boxes from the dataset. The data will only be resized
# and normalized. Keras ImageDataGenerator will be used for loading the dataset
def load_dataset(dataset_path, batch_size, image_shape):
    dataset_generator = ImageDataGenerator()
    dataset_generator = dataset_generator.flow_from_directory(
        dataset_path, target_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size,
        class_mode=None)

    return dataset_generator


# Creates the discriminator model. This model tries to classify images as real
# or fake.
def construct_discriminator(image_shape):

    discriminator = Sequential()
    discriminator.add(Conv2D(filters=64, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform',
                             input_shape=(image_shape)))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(filters=128, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    discriminator.add(BatchNormalization(momentum=0.5))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(filters=256, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    discriminator.add(BatchNormalization(momentum=0.5))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Conv2D(filters=512, kernel_size=(5, 5),
                             strides=(2, 2), padding='same',
                             data_format='channels_last',
                             kernel_initializer='glorot_uniform'))
    discriminator.add(BatchNormalization(momentum=0.5))
    discriminator.add(LeakyReLU(0.2))

    discriminator.add(Flatten())
    discriminator.add(Dense(1))
    discriminator.add(Activation('sigmoid'))

    optimizer = Adam(lr=0.0002, beta_1=0.5)
    discriminator.compile(loss='binary_crossentropy',
                          optimizer=optimizer,
                          metrics=None)

    return discriminator


# Creates the generator model. This model has an input of random noise and
# generates an image that will try mislead the discriminator.
def construct_generator():

    generator = Sequential()

    generator.add(Dense(units=4 * 4 * 512,
                        kernel_initializer='glorot_uniform',
                        input_shape=(1, 1, 100)))
    generator.add(Reshape(target_shape=(4, 4, 512)))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=256, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=128, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=64, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=3, kernel_size=(5, 5),
                                  strides=(2, 2), padding='same',
                                  data_format='channels_last',
                                  kernel_initializer='glorot_uniform'))
    generator.add(Activation('tanh'))

    optimizer = Adam(lr=0.00015, beta_1=0.5)
    generator.compile(loss='binary_crossentropy',
                      optimizer=optimizer,
                      metrics=None)

    return generator


# Displays a figure of the generated images and saves them in as .png image
def save_generated_images(generated_images, epoch, batch_number):

    plt.figure(figsize=(8, 8), num=2)
    gs1 = gridspec.GridSpec(8, 8)
    gs1.update(wspace=0, hspace=0)

    for i in range(64):
        ax1 = plt.subplot(gs1[i])
        ax1.set_aspect('equal')
        image = generated_images[i, :, :, :]
        image += 1
        image *= 127.5
        fig = plt.imshow(image.astype(np.uint8))
        plt.axis('off')
        fig.axes.get_xaxis().set_visible(False)
        fig.axes.get_yaxis().set_visible(False)

    plt.tight_layout()
    save_name = 'generated images/generatedSamples_epoch' + str(
        epoch + 1) + '_batch' + str(batch_number + 1) + '.png'

    plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
    plt.pause(0.0000000001)
    plt.show()


# Main train function
def train_dcgan(batch_size, epochs, image_shape, dataset_path):
    # Build the adversarial model that consists in the generator output
    # connected to the discriminator
    generator = construct_generator()
    discriminator = construct_discriminator(image_shape)

    gan = Sequential()
    # Only false for the adversarial model
    discriminator.trainable = False
    gan.add(generator)
    gan.add(discriminator)

    optimizer = Adam(lr=0.00015, beta_1=0.5)
    gan.compile(loss='binary_crossentropy', optimizer=optimizer,
                metrics=None)

    # Create a dataset Generator with help of keras
    dataset_generator = load_dataset(dataset_path, batch_size, image_shape)

    # 11788 is the total number of images on the bird dataset
    number_of_batches = int(11788 / batch_size)

    # Variables that will be used to plot the losses from the discriminator and
    # the adversarial models
    adversarial_loss = np.empty(shape=1)
    discriminator_loss = np.empty(shape=1)
    batches = np.empty(shape=1)

    # Allo plot updates inside for loop
    plt.ion()

    current_batch = 0

    # Let's train the DCGAN for n epochs
    for epoch in range(epochs):

        print("Epoch " + str(epoch+1) + "/" + str(epochs) + " :")

        for batch_number in range(number_of_batches):

            start_time = time.time()

            # Get the current batch and normalize the images between -1 and 1
            real_images = dataset_generator.next()
            real_images /= 127.5
            real_images -= 1

            # The last batch is smaller than the other ones, so we need to
            # take that into account
            current_batch_size = real_images.shape[0]

            # Generate noise
            noise = np.random.normal(0, 1,
                                     size=(current_batch_size,) + (1, 1, 100))

            # Generate images
            generated_images = generator.predict(noise)

            # Add some noise to the labels that will be
            # fed to the discriminator
            real_y = (np.ones(current_batch_size) -
                      np.random.random_sample(current_batch_size) * 0.2)
            fake_y = np.random.random_sample(current_batch_size) * 0.2

            # Let's train the discriminator
            discriminator.trainable = True

            d_loss = discriminator.train_on_batch(real_images, real_y)
            d_loss += discriminator.train_on_batch(generated_images, fake_y)

            discriminator_loss = np.append(discriminator_loss, d_loss)

            # Now it's time to train the generator
            discriminator.trainable = False

            noise = np.random.normal(0, 1,
                                     size=(current_batch_size * 2,) +
                                     (1, 1, 100))

            # We try to mislead the discriminator by giving the opposite labels
            fake_y = (np.ones(current_batch_size * 2) -
                      np.random.random_sample(current_batch_size * 2) * 0.2)

            g_loss = gan.train_on_batch(noise, fake_y)
            adversarial_loss = np.append(adversarial_loss, g_loss)
            batches = np.append(batches, current_batch)

            # Each 50 batches show and save images
            if((batch_number + 1) % 50 == 0 and
               current_batch_size == batch_size):
                save_generated_images(generated_images, epoch, batch_number)

            time_elapsed = time.time() - start_time

            # Display and plot the results
            print("     Batch " + str(batch_number + 1) + "/" +
                  str(number_of_batches) +
                  " generator loss | discriminator loss : " +
                  str(g_loss) + " | " + str(d_loss) + ' - batch took ' +
                  str(time_elapsed) + ' s.')

            current_batch += 1

        # Save the model weights each 5 epochs
        if (epoch + 1) % 5 == 0:
            discriminator.trainable = True
            generator.save('models/generator_epoch' + str(epoch) + '.hdf5')
            discriminator.save('models/discriminator_epoch' +
                               str(epoch) + '.hdf5')

        # Each epoch update the loss graphs
        plt.figure(1)
        plt.plot(batches, adversarial_loss, color='green',
                 label='Generator Loss')
        plt.plot(batches, discriminator_loss, color='blue',
                 label='Discriminator Loss')
        plt.title("DCGAN Train")
        plt.xlabel("Batch Iteration")
        plt.ylabel("Loss")
        if epoch == 0:
            plt.legend()
        plt.pause(0.0000000001)
        plt.show()
        plt.savefig('trainingLossPlot.png')


def main():
    dataset_path = '/home/tfreitas/Datasets/CUB_200_2011/CUB_200_2011/images/'
    batch_size = 64
    image_shape = (64, 64, 3)
    epochs = 190
    train_dcgan(batch_size, epochs,
                image_shape, dataset_path)

if __name__ == "__main__":
    main()
```

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPooling2D, Flatten，Conv2DTranspose, BatchNormalization, LeakyReLU, Reshape
from keras.optimizers import SGD


#DCGAN



def buildGenerator():
  modelG = Sequential()
  #define fully connection
  modelG.add(Dense(4*4*1024, input_dim=100))
  modelG.add(BatchNormalization())
  modelG.add(LeakyReLU())
  
  #reshape linear output to cube shape
  modelG.add(Reshape((4, 4, 1024)))
  
  #define transpose convolution
  modelG.add(Conv2DTranspose(filters=512, kernel_size=5, strides=2, padding='same', data_format='channels_last'))
  modelG.add(BatchNormalization())
  modelG.add(LeakyReLU(alpha=.01))
  
  modelG.add(Conv2DTranspose(filters=256, kernel_size=(5, 5), strides=2, padding='same', data_format='channels_last'))
  modelG.add(BatchNormalization())
  modelG.add(LeakyReLU(alpha=.01))
  
  modelG.add(Conv2DTranspose(filters=128, kernel_size=(5, 5), strides=2, padding='same', data_format='channels_last'))
  modelG.add(BatchNormalization())
  modelG.add(LeakyReLU(alpha=.01))
  
  modelG.add(Conv2DTranspose(filters=3, kernel_size=(5, 5), strides=2, padding='same', activation='tanh'))
  
  optimizer = Adam(lr=.00015, beta_l=.5)
  modelG.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=None)
  
  return modelG


  
def buildDiscriminator():
  #define discriminator
  modelD = Sequentail()
  modelD.add(Conv2D(128, kernel_size=(5, 5), padding='same', strides=(2, 2) input_shape=(64, 64, 3)))
  modelD.add(LeakyReLU())
  modelD.add(Dropout(.3))
  
  modelD = Sequentail()
  modelD.add(Conv2D(256, kernel_size=(5, 5), padding='same', strides=(2, 2)))
  modelD.add(LeakyReLU(alpha=.01))
  modelD.add(Dropout(.3))
  
  modelD = Sequentail()
  modelD.add(Conv2D(512, kernel_size=(5, 5), padding='same', strides=(2, 2)))
  modelD.add(LeakyReLU(alpha=.01))
  modelD.add(Dropout(.3))
  
  modelD.add(Flatten())
  modelD.add(Dense(1))
  optimizer = Adam(lr=.0002, beta_l=.5)
  modelD.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=None)
  
  return modelD

def load_dataset(path, batch_size, image_shape):
  return

def buildDCGAN():
  generator = buildGenerator()
  discriminator = buildDiscriminator()
  
  gan = Sequential()
  discriminator.trainable = False
  gan.add(generator)
  gan.add(discriminator)
  
  optimizer = Adam(lr=.00015, beta_l=.5)
  gan.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=None)
  
  inputG = load_dataset(path, batch_size, image_shape)
  
  number_of_batches = int(11788 / batch_size)
  
  generator_loss = np.empty(shape=1)
  discriminator_loss = np.empty(shape=1)
  batches = np.empty(shape=1)
  
  current_batch = 0
  
  for epoch in range(epochs):
    
    print("Epoch " + str(epoch + 1) + "/" + str(epochs) + ":")
    
    for batch_number in range(number_of_batches):
      
      real_images = inputG.next()
      real_images /= 127.5
      real_images -= 1
      
      current_batch_size = real_images.shape[0]
      
      noise = np.random.normal(0, 1, size=(current_batch_size, 1, 1, 100))
      
      generator_images = generator.predict(noise)
      
      real_y = (np.ones(current_batch_size) - np.random.random_sample(current_batch_size) * .2)
      fake_y = np.random.random_sample(current_batch_size) * .2
      
      
      #train the discriminator
      discriminator.trainable = True
      
      d_loss = discriminator.train_on_batch(real_images, real_y)
      d_loss += discriminator.train_on_batch(generator_images, fake_y)
      
      discriminator_loss = np.append(discriminator_loss, d_loss)
      
      discriminator.trainable = False
      
      noise = np.random.normal(0, 1, size=(current_batch_size * 2, 1, 1, 100))
      
      fake_y = (np.ones(current_batch_size * 2) - np.random.random_sample(current_batch_size * 2) * .2)
      
      g_loss = gan.train_on_batch(noise, fake_y)
      generator_loss = np.append(generator_loss, g_loss)
      batches = np.append(batches, current_batch)
      
      if((batch_number + 1) % 50 == 0 and current_batch_size == batch_size):
        save_generated_image(generator_images, epoch, batch_number)
      
      
  

             
  
  
  
  
  
  